In [2]:
import sys
sys.path.append('..')

import constants.file_handler_constants as fh
from constants.user_constants import *
from constants.attraction_constants import *
from packages.user_preference_prompt_by_persona.gernerate_preference import *

import os
import glob
import time
import pandas as pd
import numpy as np
import json
import requests
import ast
import random as rd


In [10]:
# keep save id by following result to constant (manually)

user_id_love_nature = []
user_id_love_art = []
user_id_love_temple_historical = []
user_id_love_adventure = []
user_id_love_beach = []
user_id_love_family = []
user_id_love_citylife = []

try:
    API_ENDPOINT = f"{os.environ['TRIPWEAVER_API']}/api/user/getAllUser"
    headers = {
        "Authorization": f"{os.environ['AUTH_BRANCH_API_KEY']}",
        "Content-Type": "application/json"
    }
    res_all_user = requests.get(url=API_ENDPOINT, headers=headers).json()
    user_id_love_nature = [cur_user['_id'] for cur_user in res_all_user if ("love_nature_" in cur_user['username'])] 
    user_id_love_art = [cur_user['_id'] for cur_user in res_all_user if ("love_art_" in cur_user['username'])] 
    user_id_love_temple_historical = [cur_user['_id'] for cur_user in res_all_user if ("love_temple_historical_" in cur_user['username'])]
    user_id_love_adventure = [cur_user['_id'] for cur_user in res_all_user if ("love_adventure_" in cur_user['username'])]
    user_id_love_beach = [cur_user['_id'] for cur_user in res_all_user if ("love_beach_" in cur_user['username'])]
    user_id_love_family = [cur_user['_id'] for cur_user in res_all_user if ("love_family_" in cur_user['username'])]
    user_id_love_citylife = [cur_user['_id'] for cur_user in res_all_user if ("love_citylife_" in cur_user['username'])]


except Exception as e:
    print("get user failed !")
    print(e)

print("user_id_love_nature :\n", user_id_love_nature)
print("user_id_love_temple_historical :\n", user_id_love_temple_historical)
print("user_id_love_adventure :\n", user_id_love_adventure)
print("user_id_love_art :\n", user_id_love_art)
print("user_id_love_beach  :\n", user_id_love_beach)
print("user_id_love_family :\n", user_id_love_family)
print("user_id_love_citylife :\n", user_id_love_citylife)

user_id_love_nature :
 ['677e41b2519209b0aaff4d76', '677e41b6519209b0aaff4d7b', '677e41b9519209b0aaff4d80', '677e41bc519209b0aaff4d85', '677e41bf519209b0aaff4d8a', '677e41c2519209b0aaff4d8f', '677e41c5519209b0aaff4d94', '677e41c8519209b0aaff4d99', '677e41cb519209b0aaff4d9e', '677e41ce519209b0aaff4da3', '677e41d1519209b0aaff4da8', '677e41d4519209b0aaff4dad', '677e41d7519209b0aaff4db2', '677e41da519209b0aaff4db7', '677e41de519209b0aaff4dbc', '677e41df519209b0aaff4dc1', '677e41e1519209b0aaff4dc6', '677e41e2519209b0aaff4dcb', '677e41e3519209b0aaff4dd0', '677e41e5519209b0aaff4dd5', '6799d5c5dd4c73b429f7a4c8', '6799d5c8dd4c73b429f7a4cd', '6799d5cbdd4c73b429f7a4d2', '6799d5cfdd4c73b429f7a4d7', '6799d5d3dd4c73b429f7a4dc', '6799d5d6dd4c73b429f7a4e1', '6799d5d9dd4c73b429f7a4e6', '6799d5dddd4c73b429f7a4eb', '6799d5e0dd4c73b429f7a4f0', '6799d5e4dd4c73b429f7a4f5', '6799d5e7dd4c73b429f7a4fa', '6799d5ebdd4c73b429f7a4ff', '6799d5eedd4c73b429f7a504', '6799d5f2dd4c73b429f7a509', '6799d5f6dd4c73b429f7a50

In [12]:
''' 
50 rating of each user in following persona(350 users) have been created to same type attractions:
- user_persona_nature --> love_art_{nth}
- user_persona_temple_historical --> love_temple_historical_{nth}
- user_persona_adventure --> love_adventure_{nth}
- user_persona_art --> love_art_{nth}
- user_persona_beach --> love_beach_{nth}
- user_persona_family --> love_family_{nth}
- user_persona_citylife --> love_citylife_{nth}
'''

# select both the corresponding group of user_id and attraction_id in 'user_constants'
selected_persona_users_id = USER_ID_LOVE_CITYLIFE
selected_persona_attractions_id = ATTRACTION_ID_LOVE_CITYLIFE

for cur_user_id in selected_persona_users_id:
    for cur_attraction_id in selected_persona_attractions_id:
        try:
            API_ENDPOINT = f"{os.environ['TRIPWEAVER_API']}/api/userrating/patch/{cur_user_id}"
            cur_payload = {
                "attractionId": cur_attraction_id, 
                "rating_score": rd.randint(4, 5)
            }
            res_user_rating = requests.patch(url=API_ENDPOINT, json=cur_payload).json()

        except Exception as e:
            print("get user_rating failed !")
            print(e)


In [44]:
# mix persona rating with others types of attrations
# to make collaborative data more realistic.

persona_users_id = USER_ID_LOVE_TEMPLE_HISTORICAL
related_persona_attractions_id = ATTRACTION_ID_LOVE_TEMPLE_HISTORICAL

def flatten_concatenation_rand_half(matrix):
    '''
    Flatten a group of lists,
    where each element of the list is randomly selected (half of all elements in each sublist).
    '''
    flat_list = []
    for row in matrix:
        len_half = len(row)//2 
        flat_list += rd.sample(row, len_half)
    return flat_list

group_persona_attractions_id = [
    ATTRACTION_ID_LOVE_NATURE, ATTRACTION_ID_LOVE_TEMPLE_HISTORICAL, ATTRACTION_ID_LOVE_ADVENTURE, 
    ATTRACTION_ID_LOVE_ART, ATTRACTION_ID_LOVE_BEACH, ATTRACTION_ID_LOVE_FAMILY, ATTRACTION_ID_LOVE_CITYLIFE
]

related_persona_pos = group_persona_attractions_id.index(related_persona_attractions_id)

group_persona_attractions_id.pop(related_persona_pos)

for cur_user_id in persona_users_id:
    rand_group_persona_attractions_id = rd.sample(group_persona_attractions_id, 3)
    flatten_rand_group_persona_attractions_id = flatten_concatenation_rand_half(rand_group_persona_attractions_id)
    for cur_attraction_id in flatten_rand_group_persona_attractions_id:
        try:
            API_ENDPOINT = f"{os.environ['TRIPWEAVER_API']}/api/userrating/patch/{cur_user_id}"
            cur_payload = {
                "attractionId": cur_attraction_id, 
                "rating_score": rd.randint(3, 5)
            }
            res_user_rating = requests.patch(url=API_ENDPOINT, json=cur_payload).json()
            
        except Exception as e:
            print("get user_rating failed !")
            print(e)
